## Research ideas


- High dimensional inner products $\langle x, y \rangle$. If x, y are both in TT-format then it can be done 
- Tensorized NN with rank dropout

In [58]:
!pip install tntorch

  Using cached tntorch-1.1.2-py3-none-any.whl.metadata (998 bytes)
Using cached tntorch-1.1.2-py3-none-any.whl (63 kB)


In [ ]:
from typing import List, Union, Optional
import torch
import tntorch as tn

def mpo_contract(
    mpo: Union[List[torch.Tensor], torch.nn.ParameterList], 
    mps: Union[List[torch.Tensor], torch.Tensor]
    ) -> Union[List[torch.Tensor], torch.Tensor]: # returns a new mps
    """Perform a tensor network contraction of an MPO and an MPS. Returns a new MPS.

    Args:
        mpo (List[torch.Tensor]): A list of tensors representing the MPO. Shape: (Rl, Di, Do, Rr)
        mps (List[torch.Tensor]): A list of tensors representing the MPS. Shape: (B, Rl, Di, Rr)

    Returns:
        List[torch.Tensor]: A list of tensors representing the new MPS. Shape: (B, Do, Rl, Rr)
    """
    out = []
    for i in range(len(mpo)):
        mps_prime = torch.einsum('rios,bpiq->brposq', mps[i], mpo[i])
        B, R, P, Do, S, Q = mps_prime.shape
        out.append(mps_prime.reshape(B, R*P, Do, S*Q))
    return out


class MPO(torch.nn.Module):
    def __init__(self, in_features: List[int], out_features: List[int], ranks: List[int], max_rank: int = 2):
        super(MPO, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.ranks = ranks
        self.max_rank = max_rank
        self.g = torch.nn.ParameterList([
            torch.nn.Parameter(torch.randn(ranks[i], in_features[i], out_features[i], ranks[i+1]))
            for i in range(len(in_features))  # (Rk, Ik, Ok, Rk+1)
        ])

    def forward(self, mps_x: List[torch.Tensor], mps_y: Optional[List[torch.Tensor]] = None) -> torch.Tensor:
        # mps_x:[(B, Rl, Di, Rr), ...], mps_y:[(B, Rl, Do, Rr), ...]

        # MPO x MPS
        mps_y_hat = mpo_contract(self.g, mps_x)  

        # Rank dropout
        mps_y_hat_reduced = []
        for i in range(len(mps_y_hat)):
            # print(mps_y_hat[i].shape, self.max_rank)
            Rl = torch.randint(0, mps_y_hat[i].shape[1], (self.max_rank,))  # (B, Rl, Do, Rr)
            Rr = torch.randint(0, mps_y_hat[i].shape[3], (self.max_rank,))
            # print(Rl, Rr)
            mps_y_hat_reduced.append(mps_y_hat[i][:, Rl][:, :, :, Rr])

        # Non-linearity
        mps_y_hat_reduced = [torch.relu(mps_y_hat_reduced[i]) for i in range(len(mps_y_hat_reduced))]
        
        return mps_y_hat_reduced

class TNN(torch.nn.Module):
    def __init__(self, in_features: List[int], out_features: List[int], n_layers: int=4, max_rank: int = 2):
        super(TTN, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.n_layers = n_layers
        self.max_rank = max_rank
        self.layers = torch.nn.ModuleList([
            MPO(
                in_features=in_features, 
                out_features=out_features, 
                ranks=[2] * (n_layers + 1), 
                max_rank=max_rank
            ) for _ in range(n_layers)
        ])
    
    def forward(self, mps_x: List[torch.Tensor], mps_y: Optional[List[torch.Tensor]] = None) -> torch.Tensor:
        for i, layer in enumerate(self.layers):
            mps_x = layer(mps_x, mps_y)
        return mps_x



# Test MPO
# in_features = [2, 2]
# out_features = [2, 2]
# ranks = [2, 2, 2]
# batch_size = 2
# mpo = MPO(in_features=in_features, out_features=out_features, ranks=ranks)
# mps = [torch.randn(batch_size, ranks[i], in_features[i], ranks[i+1]) for i in range(len(in_features))]
# result = mpo(mps)
# print(f"Rank: " + ', '.join([str(r.shape) for r in result]))

# Test TTN
batch_size = 2
ranks = [2, 2, 2]
in_features = [2, 2]
out_features = [2, 2]
n_layers = 8
max_rank = 2
mps = [torch.randn(batch_size, ranks[i], in_features[i], ranks[i+1]) for i in range(len(in_features))]
ttn = TNN(in_features=in_features, out_features=out_features, n_layers=n_layers, max_rank=max_rank)
result = ttn(mps)
print(f"Rank: " + ', '.join([str(r.shape) for r in result]))

Layer 1 done
Layer 2 done
Layer 3 done
Layer 4 done
Layer 5 done
Layer 6 done
Layer 7 done
Layer 8 done
Rank: torch.Size([2, 2, 2, 2]), torch.Size([2, 2, 2, 2])


In [96]:
i = 0
max_rank=2
mps_y_hat=mpo_contract(mpo.g, mps)  
Rl = torch.randint(0, mps_y_hat[i].shape[1], (max_rank,))
Rr = torch.randint(0, mps_y_hat[i].shape[3], (max_rank,))
# mps_y_hat[i].shape
Rl.shape, mps_y_hat[i].shape, mps_y_hat[i][:, :, :, Rr][:, Rl].shape

(torch.Size([2]), torch.Size([2, 4, 2, 4]), torch.Size([2, 2, 2, 2]))

In [66]:
# ranks = [2, 2, 2]
# dims = [2, 2]
# tt_cores = [torch.randn(ranks[i], dims[i], ranks[i+1]) for i in range(len(dims))]
mps_x = [torch.randn(ranks[i], in_features[i], ranks[i+1]) for i in range(len(in_features))]
tt_tens = tn.tensor.Tensor(mps_x)
# tt_tens
# mps_y_hat = mpo_contract(mpo.g, mps_x)


In [ ]:
import torch, tensorkrowch as tk
torch.manual_seed(0)

# MPO contraction
batch_size, n_features, in_dim, out_dim, bond_dim = 8, 5, 2, 2, 5
mpo = tk.models.MPO(n_features=n_features,
                   in_dim=in_dim,
                   out_dim=out_dim,
                   bond_dim=bond_dim)
data = torch.ones(batch_size, n_features, in_dim) # batch_size x n_features x feature_size
result = mpo(data)  
print(result.shape)



torch.Size([8, 2, 2, 2, 2, 2])


In [ ]:
import torch, tensorkrowch as tk
torch.manual_seed(0)

# MPO X MPSData
batch_size, n_features, in_dim, out_dim, bond_dim = 8, 5, 2, 2, 5
mpo = tk.models.MPO(n_features=n_features,
                   in_dim=in_dim,
                   out_dim=out_dim,
                   bond_dim=bond_dim)
data = torch.ones(batch_size, n_features, in_dim) # batch_size x n_features x feature_size
mps_data = tk.models.MPSData(n_features=n_features,
                         phys_dim=in_dim,
                         bond_dim=bond_dim,
                         boundary="pbc"
                        )
data = [torch.ones(batch_size, bond_dim, in_dim, bond_dim) for _ in range(n_features)]
mps_data.add_data(data)
result = mpo.contract(mps=mps_data)  
result

In [31]:
!pip install git+https://github.com/yastn/yastn

  Cloning https://github.com/yastn/yastn to /private/var/folders/r_/d81gvkws1n5fg2_7mb57cwzh0000gn/T/pip-req-build-7pkdhf1l
  Running command git clone --filter=blob:none --quiet https://github.com/yastn/yastn /private/var/folders/r_/d81gvkws1n5fg2_7mb57cwzh0000gn/T/pip-req-build-7pkdhf1l
  Resolved https://github.com/yastn/yastn to commit 4670be709907863042aa968b2e5014529a1ca524
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached h5py-3.14.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.7 kB)
Using cached h5py-3.14.0-cp310-cp310-macosx_11_0_arm64.whl (2.8 MB)
  Created wheel for yastn: filename=yastn-1.6.1-py3-none-any.whl size=295809 sha256=b828001233396732ed43dd1846597dfadb5aa856b9fb820325abaa1233608c3b
  Stored in directory: /private/var/folders/r_/d81gvkws1n5fg2_7mb57cwzh0000gn/T/pip-ephem-wheel-cache-46580chp/wheels/78/6b/0d/77992f21fadd55f05fdc6060797ea74c7528cb535489703bef
Successf

In [37]:
import yastn
import yastn.tn.mps as mps
import yastn.operators as ops

# Use PyTorch backend
cfg = yastn.make_config(backend='torch')

# Build local identity operator and a trivial product-MPO to set spaces
Id = ops.Qdit(d=2, **cfg._asdict()).I()
I_mpo = mps.product_mpo(Id, N=8)          # defines geometry/phys dims

# Random MPS and random MPO (both compatible with I_mpo spaces)
psi  = mps.random_mps(I_mpo, D_total=8)   # MPS
O    = mps.random_mpo(I_mpo, D_total=6)   # MPO

# 1) Exact product: MPO @ MPS -> MPS
phi = O @ psi

# 2) Compressed application (zipper) with SVD truncation
phi_zip = mps.zipper(O, psi, opts_svd={'max_truncation_err': 1e-8})

# Suppose psi_target is another MPS with same geometry
phi_target = mps.random_mps(I_mpo, D_total=8)   # MPS

# MSE = ||phi - psi_target||² / N
diff = phi - phi_target
diff


In [46]:
"""
Tensor Train multiplication: TT-Matrix × TT-Vector → TT-Vector
Pure tensor train operations with random initialization
"""

import numpy as np

def random_tt_vector(d, n, r_max):
    """
    Create random TT-vector (rank-3 tensors).
    
    Args:
        d: physical dimension at each site
        n: number of sites (chain length)
        r_max: maximum bond/rank dimension
    
    Returns:
        List of cores: [G[0], G[1], ..., G[n-1]]
        where G[i] has shape (r[i], d, r[i+1])
    """
    cores = []
    r_prev = 1
    
    for i in range(n):
        if i == n - 1:
            r_next = 1
        else:
            r_next = min(r_max, d**(min(i+1, n-i-1)))
        
        core = np.random.randn(r_prev, d, r_next)
        cores.append(core)
        r_prev = r_next
    
    return cores

def random_tt_matrix(d_in, d_out, n, r_max):
    """
    Create random TT-matrix (rank-4 tensors).
    
    Args:
        d_in: input physical dimension
        d_out: output physical dimension
        n: number of sites
        r_max: maximum bond/rank dimension
    
    Returns:
        List of cores: [W[0], W[1], ..., W[n-1]]
        where W[i] has shape (r[i], d_out, d_in, r[i+1])
    """
    cores = []
    r_prev = 1
    
    for i in range(n):
        if i == n - 1:
            r_next = 1
        else:
            r_next = min(r_max, (d_in*d_out)**(min(i+1, n-i-1)))
        
        core = np.random.randn(r_prev, d_out, d_in, r_next)
        cores.append(core)
        r_prev = r_next
    
    return cores

def tt_matrix_vector_product(W_cores, G_cores, r_max=None):
    """
    Compute TT-matrix × TT-vector product.
    
    Args:
        W_cores: TT-matrix cores, each shape (r_W[i], d_out, d_in, r_W[i+1])
        G_cores: TT-vector cores, each shape (r_G[i], d_in, r_G[i+1])
        r_max: max rank for compression (None = no compression)
    
    Returns:
        Result TT-vector cores, each shape (r[i], d_out, r[i+1])
    """
    n = len(G_cores)
    result_cores = []
    
    for i in range(n):
        W = W_cores[i]  # (r_W, d_out, d_in, r_W')
        G = G_cores[i]  # (r_G, d_in, r_G')
        
        # Contract over d_in dimension
        # W: (r_W, d_out, d_in, r_W') × G: (r_G, d_in, r_G')
        # Result: (r_W, d_out, r_W', r_G, r_G')
        # W: (1, 4, 4, 5), G: (1, 4, 4)
        print(f"W: {W.shape}, G: {G.shape}")
        temp = np.einsum('abcd,ece->abdce', W, G)
        
        # Reshape: merge left bonds (r_W, r_G) and right bonds (r_W', r_G')
        r_W, d_out, r_W_next, r_G, r_G_next = temp.shape
        temp = temp.reshape(r_W * r_G, d_out, r_W_next * r_G_next)
        
        result_cores.append(temp)
    
    # Optional: compress using SVD
    if r_max is not None:
        result_cores = compress_tt(result_cores, r_max)
    
    return result_cores

def compress_tt(cores, r_max):
    """Compress TT using SVD (left-to-right sweep)."""
    n = len(cores)
    compressed = []
    
    for i in range(n - 1):
        core = cores[i]
        r_left, d, r_right = core.shape
        
        # Reshape to matrix
        mat = core.reshape(r_left * d, r_right)
        
        # SVD
        U, S, Vt = np.linalg.svd(mat, full_matrices=False)
        
        # Truncate
        r_trunc = min(r_max, len(S))
        U = U[:, :r_trunc]
        S = S[:r_trunc]
        Vt = Vt[:r_trunc, :]
        
        # Store left part
        compressed.append(U.reshape(r_left, d, r_trunc))
        
        # Merge S*Vt into next core
        cores[i + 1] = np.einsum('ij,jkl->ikl', S[:, None] * Vt, cores[i + 1])
    
    compressed.append(cores[-1])
    return compressed

def tt_ranks(cores):
    """Get bond dimensions of TT."""
    return [cores[i].shape[2] for i in range(len(cores) - 1)]

# Example usage
print("Tensor Train Matrix × Vector Product")
print("=" * 60)

# Parameters
n = 8           # number of sites
d = 4           # physical dimension
r_max_init = 5  # initial max rank
r_max_result = 10  # max rank for result

print(f"\nTensor train length: {n}")
print(f"Physical dimension: {d}")
print(f"Max rank: {r_max_init}")

# Create random TT-vector (initial state)
print("\n1. Creating random TT-vector...")
G_cores = random_tt_vector(d, n, r_max_init)
print(f"   TT-vector ranks: {tt_ranks(G_cores)}")

# Create random TT-matrix (operator)
print("\n2. Creating random TT-matrix...")
W_cores = random_tt_matrix(d, d, n, r_max_init)
print(f"   TT-matrix ranks: {tt_ranks(W_cores)}")

# Perform multiplication
print("\n3. Computing TT-matrix × TT-vector...")
result_cores = tt_matrix_vector_product(W_cores, G_cores, r_max=r_max_result)
print(f"   Result TT-vector ranks: {tt_ranks(result_cores)}")

# print("\n4. Tensor shapes:")
# print(f"   Input vector cores:  {[c.shape for c in G_cores[:3]]} ...")
# print(f"   Matrix cores:        {[c.shape for c in W_cores[:3]]} ...")
# print(f"   Output vector cores: {[c.shape for c in result_cores[:3]]} ...")

# print("\n✓ Done! Result is a compressed TT-vector.")
# print("\nNote: This is pure tensor train algebra - no physics assumptions.")

Tensor Train Matrix × Vector Product

Tensor train length: 8
Physical dimension: 4
Max rank: 5

1. Creating random TT-vector...
   TT-vector ranks: [4, 5, 5, 5, 5, 5, 4]

2. Creating random TT-matrix...
   TT-matrix ranks: [4, 4, 4, 4, 4, 4, 4]

3. Computing TT-matrix × TT-vector...
W: (1, 4, 4, 5), G: (1, 4, 4)


ValueError: dimensions in operand 1 for collapsing index 'e' don't match (1 != 4)

In [48]:
!pip install cotengra

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 15.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [cotengra]


In [51]:
import cotengra as ctg

x = torch.randn(10, 10)
y = torch.randn(10, 10)

# einsum style
z = ctg.einsum("ab,bc->ca", x, y)

# programmatic style
z = ctg.array_contract(
  arrays=(x, y),
  inputs=[(0, 1), (1, 2)],
  output=(2, 0),
)

In [ ]:
mps_rl, mps_rr = 3, 3
mpo_rl, mpo_rr = 3, 3
N, Di, Do = 10, 2, 2
mps = [torch.randn(Do, Di, 1, mpo_rr)] + [torch.randn(Do, Di, mps_rl, mpo_rr) for _ in range(N-2)] + [torch.randn(Do, Di, mpo_rl, 1)]
mpo = [torch.randn(Di, mps_rl, mps_rr)] +[torch.randn(Di, mps_rl, mps_rr) for _ in range(N-2)] + [torch.randn(Di, mps_rl, mps_rr)]
# do, di,  rr, rl
inputs_mpo = [[n, N+n, 2*N+n - 1, 2*N+n] for n in range(1,N+1)]
# di, rl, rr
inputs_mps = [[N+n, 3*N+n - 2 , 3*N+n -1] for n in range(1,N+1)]

outputs = [[n] for n in range(1,N+1)]

# einsum style
z = ctg.einsum("ab,bc->ca", x, y)

# programmatic style
z = ctg.array_contract(
  arrays=mps + mpo,
  inputs=inputs_mps + inputs_mpo,
  output=(2, 0),
)